In [1]:
import os
import numpy as np
import cv2 as cv
  

#takes the label file folder and gives the coordinate-suffixes for moving the aerial photos to the correct folder for cnn 
def labelData(label_data_path):
    os.chdir(label_data_path)
    true = []
    false = []
    for file in os.listdir():
        img = cv.imread(file)
        b, g, r = cv.split(img)
        mask = r < 254
        mask = mask.astype(int)
        if np.count_nonzero(mask) > img.shape[0]*img.shape[1] / 2:
            true.append(file[11:])
        else: 
            false.append(file[11:])
    return true, false

def moveDataByLabels(true, false, source_path, target_path, dataset):
    for t in true:
        os.rename(f"{source_path}/{dataset}_{t}", f"{target_path}/true/{dataset}_{t}")
    for f in false:
        os.rename(f"{source_path}/{dataset}_{f}", f"{target_path}/false/{dataset}_{f}")

label_data_path = ""
source_path = ""
target_path = ""

true, false = labelData(label_data_path)
moveDataByLabels(true, false, source_path, target_path, "avoindata:Ortoilmakuva_2019_20cm")



In [2]:
import torch
import torch.utils.data
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F

BATCH_SIZE_TRAIN = 100
BATCH_SIZE_TEST = 100
LEARNING_RATE = 0.01
NO_EPOCHS = 3

/home/tulijoki/Data/Project/code/gisenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
train_data_path = ""
test_data_path = ""

transform = transforms.Compose([transforms.ToTensor()])

train_set = datasets.ImageFolder(train_data_path, transform=transform)
test_set = datasets.ImageFolder(test_data_path, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=BATCH_SIZE_TRAIN, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=BATCH_SIZE_TEST, shuffle=False)


In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
          nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1), 
          nn.ReLU(),
          nn.MaxPool2d(2,2), 
          nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3, padding=1), 
          nn.ReLU(),
          nn.MaxPool2d(2,2),  
          nn.Flatten(),
          nn.Linear(131072, 1024),
          nn.Linear(1024, 32),
          nn.Linear(32,2)
        )

    def forward(self, x):
        return self.model(x)

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

model = CNN().to(device)

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), LEARNING_RATE)

for epoch in range(NO_EPOCHS):
    train_loss = 0
    train_correct = 0
    total = 0
    for batch_num, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        predicted = model(data)
        predicted_class = torch.argmax(predicted, dim=1)
        train_correct += BATCH_SIZE_TRAIN - torch.count_nonzero(predicted_class - target)
        loss = loss_function(predicted, target)
        total += BATCH_SIZE_TRAIN
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    print(f"Training: Epoch {epoch}, Loss: {train_loss}, Accuracy: {train_correct / total}")


Training: Epoch 0, Loss: 91.56700211763382, Accuracy: 0.7554545402526855
Training: Epoch 1, Loss: 77.53475990891457, Accuracy: 0.8156060576438904
Training: Epoch 2, Loss: 73.07332594692707, Accuracy: 0.8321717381477356


In [6]:
with torch.no_grad():
    for batch_num, (data, target) in enumerate(test_loader):
        total = BATCH_SIZE_TEST
        data, target = data.to(device), target.to(device)
        predicted = model(data)
        predicted_class = torch.argmax(predicted, dim=1)
        diff = predicted_class - target
        test_correct = BATCH_SIZE_TEST - torch.count_nonzero(diff)
        loss = loss_function(predicted, target)
        test_loss = loss.item()

        print(f"Testing: Batch: {batch_num} Loss: {test_loss}, Accuracy: {test_correct / total}")

Testing: Batch: 0 Loss: 0.323301762342453, Accuracy: 0.8700000047683716
Testing: Batch: 1 Loss: 0.2725062966346741, Accuracy: 0.9200000166893005
